Kütüphanelerin Eklenmesi

In [123]:
import pandas as pd           
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMRegressor

In [124]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Veri

In [125]:
training = pd.read_csv("training.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

Sütunlar arasındaki ilişkilerin incelenmesi sonucunda hedef sütununa yararı olmayan sütunların veri setinden çıkarılması

In [126]:
columns = [
    'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 
    'income_household_under_5', 'income_household_5_to_10', 'income_household_10_to_15', 'income_household_15_to_20', 
    'income_household_20_to_25', 'income_household_25_to_35', 'income_household_35_to_50', 'income_household_50_to_75', 
    'income_household_75_to_100', 'income_household_100_to_150', 'income_household_150_over', 'education_less_highschool', 
    'education_highschool', 'education_some_college', 'education_bachelors', 'education_graduate', 'education_college_or_above', 
    'education_stem_degree', 'married', 'divorced', 'never_married', 'widowed', 'male', 'female', 'home_ownership', 
    'housing_units', 'home_value', 'rent_median', 'rent_burden', 'age_median', 'farmer', 'bmi', 
    'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type', 'patient_race', 'payer_type', 'patient_gender'
]
training = training.drop(columns, axis=1)
test = test.drop(columns, axis=1)

train ve test veri setlerindeki metin verisi bulunan sütunlardaki mantık hatalarını düzeltme

In [127]:
merged_data = pd.concat([training, test], axis=0)

In [128]:
merged_data.loc[merged_data["breast_cancer_diagnosis_desc"] == 'Malignant neoplasm of unspecified site of right male breast', 'breast_cancer_diagnosis_desc'] = 'Malignant neoplasm of unspecified site of right female breast'
merged_data.loc[merged_data["breast_cancer_diagnosis_desc"] == 'Malignant neoplasm of unspecified site of left male breast', 'breast_cancer_diagnosis_desc'] = 'Malignant neoplasm of unspecified site of left female breast'
merged_data.loc[merged_data['breast_cancer_diagnosis_code'] == 'C50922', 'breast_cancer_diagnosis_code'] = 'C50912'
merged_data.loc[merged_data["breast_cancer_diagnosis_desc"] == 'Malig neoplasm of upper-outer quadrant of right male breast', 'breast_cancer_diagnosis_desc'] = 'Malig neoplasm of upper-outer quadrant of right female breast'
merged_data.loc[merged_data["breast_cancer_diagnosis_desc"] == 'Malignant neoplasm of other and unspecified sites of male breast', 'breast_cancer_diagnosis_desc'] = 'Malignant neoplasm of other and unspecified sites of female breast'
merged_data.loc[merged_data["breast_cancer_diagnosis_desc"] == 'Malignant neoplasm of nipple and areola, right male breast', 'breast_cancer_diagnosis_desc'] = 'Malignant neoplasm of nipple and areola, right female breast'
merged_data.loc[merged_data['breast_cancer_diagnosis_code'] == 'C50021', 'breast_cancer_diagnosis_code'] = 'C50011'
merged_data.loc[merged_data["breast_cancer_diagnosis_desc"] == 'Malignant neoplasm of unsp site of unspecified male breast', 'breast_cancer_diagnosis_desc'] = 'Malignant neoplasm of unsp site of unspecified female breast'
merged_data.loc[merged_data['breast_cancer_diagnosis_code'] == 'C50929', 'breast_cancer_diagnosis_code'] = 'C50919'

In [129]:
training = merged_data.iloc[:len(training), :]
test = merged_data.iloc[len(training):, :]

'patient_state', 'Region', 'Division' sütunlarındaki boş hücreleri doldurma

In [130]:
dropped_columns = [
    'population', 'density', 'family_size', 'family_dual_income', 'income_household_median', 
    'income_household_six_figure', 'income_individual_median', 'labor_force_participation',
    'unemployment_rate', 'self_employed', 'race_white', 'race_black', 'race_asian', 
    'race_native', 'race_pacific', 'race_other', 'race_multiple', 'hispanic', 'disabled', 
    'poverty', 'limited_english', 'commute_time', 'health_uninsured', 'veteran', 'Ozone', 
    'PM25', 'N02'
]

In [131]:
a = training[dropped_columns]

In [132]:
training = training.drop(dropped_columns, axis=1)

In [133]:
c = training 

In [134]:
len(c[c.isnull().any(axis=1)])

52

In [135]:
b = pd.DataFrame()
b = c[c.isnull().any(axis=1)]
print(len(b))

52


In [136]:
rows = training[c.isnull().any(axis=1)]
unique_null_zip3_numbers = rows["patient_zip3"].unique()
unique_null_zip3_numbers

array([973, 330, 140, 916, 117, 853, 181, 111, 322, 136, 331, 662, 148,
       300, 347, 711, 953, 975, 660, 324, 906, 338, 601, 420, 972, 974,
       454, 900, 779, 913, 125, 157, 857, 142, 968, 902, 970, 294, 334,
       666, 781, 335, 930, 109], dtype=int64)

In [137]:
c = c.dropna()

In [138]:
for i in unique_null_zip3_numbers:
    rows2 = c[c['patient_zip3'] == i]
    if i in rows2['patient_zip3'].unique():
        null_mask = b["patient_zip3"] == i
        null_rows = b[null_mask][b[null_mask].isnull().any(axis=1)]
        b.loc[null_mask, "patient_state"] = rows2["patient_state"].unique()[0]
        b.loc[null_mask, "Region"] = rows2["Region"].unique()[0]
        b.loc[null_mask, "Division"] = rows2["patient_state"].unique()[0]

In [139]:
training = pd.concat([training.drop(b.index), b])

In [140]:
for column_name in dropped_columns:
    training[column_name] = a[column_name]

'N02', 'Ozone', 'PM25' sütunlarındaki boş hücreleri doldurma

In [141]:
mean_N02 = training["N02"].mean()
mean_Ozone = training["Ozone"].mean()
mean_PM25 = training["PM25"].mean()

In [142]:
training["N02"] = training["N02"].fillna(value=mean_N02)
training["Ozone"] = training["Ozone"].fillna(value=mean_Ozone)
training["PM25"] = training["PM25"].fillna(value=mean_PM25)

Boş hücre bulunan satırları doldurma

In [143]:
f = training[training.isnull().any(axis=1)]

In [144]:
training[training["patient_state"] == 'TX']["family_size"].mean()

3.324796447735702

In [145]:
columns2 = [
    "family_size", "family_dual_income", "income_household_median", 
    "income_household_six_figure", "self_employed", "poverty", "limited_english"
]
for i in columns2:
    training.at[2996, i] = training[training["patient_state"] == 'TX'][i].mean()
    training.at[8481, i] = training[training["patient_state"] == 'TX'][i].mean()
    training.at[10542, i] = training[training["patient_state"] == 'TX'][i].mean()
    training.at[11314, i] = training[training["patient_state"] == 'TX'][i].mean()

In [146]:
training[training.isnull().any(axis=1)]

,patient_id,patient_state,patient_zip3,patient_age,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,Region,Division,DiagPeriodL90D,population,density,family_size,family_dual_income,income_household_median,income_household_six_figure,income_individual_median,labor_force_participation,unemployment_rate,self_employed,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
10542,224030,FL,332,41,C50911,Malignant neoplasm of unsp site of right femal...,C7800,South,South Atlantic,0.0,NaN,NaN,3.324796,50.836314,69032.323663,29.640414,NaN,NaN,NaN,14.272299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.88433,7.130051,NaN,NaN,NaN,35.544993,8.714016,16.558153


In [147]:
training[training["patient_zip3"] == 332]

,patient_id,patient_state,patient_zip3,patient_age,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,Region,Division,DiagPeriodL90D,population,density,family_size,family_dual_income,income_household_median,income_household_six_figure,income_individual_median,labor_force_participation,unemployment_rate,self_employed,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
10542,224030,FL,332,41,C50911,Malignant neoplasm of unsp site of right femal...,C7800,South,South Atlantic,0.0,NaN,NaN,3.324796,50.836314,69032.323663,29.640414,NaN,NaN,NaN,14.272299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.88433,7.130051,NaN,NaN,NaN,35.544993,8.714016,16.558153


In [148]:
columns3 = [
    "population", "density", "income_individual_median", "labor_force_participation", 
    "unemployment_rate", "race_white", "race_black", "race_asian", "race_native", 
    "race_pacific", "race_other", "race_multiple", "hispanic", "disabled",
    "commute_time", "health_uninsured", "veteran"
]
for i in columns3:
    training.at[10542, i] = training[training["patient_state"] == 'FL'][i].mean()

In [149]:
test = test.drop(['DiagPeriodL90D'], axis=1)

In [150]:
dropped_columns_2 = [
    'family_size', 'family_dual_income', 'income_household_median', 
    'income_household_six_figure', 'self_employed', 'poverty', 'limited_english', 
    'Ozone', 'PM25', 'N02'
]

In [151]:
a = test[dropped_columns_2]

In [152]:
test = test.drop(dropped_columns_2, axis=1)

In [153]:
c = test

In [154]:
len(c[c.isnull().any(axis=1)])

21

In [155]:
b = pd.DataFrame()
b = c[c.isnull().any(axis=1)]
print(len(b))

21


In [156]:
rows = test[c.isnull().any(axis=1)]
unique_null_zip3_numbers = rows["patient_zip3"].unique()
unique_null_zip3_numbers

array([975, 937, 303, 390, 902, 974, 705, 451, 785, 852, 394, 130, 986,
       112, 473, 750, 645], dtype=int64)

In [157]:
c = c.dropna()

In [158]:
for i in unique_null_zip3_numbers:
    rows2 = c[c['patient_zip3'] == i]
    if i in rows2['patient_zip3'].unique():
        null_mask = b["patient_zip3"] == i
        null_rows = b[null_mask][b[null_mask].isnull().any(axis=1)]
        b.loc[null_mask, "patient_state"] = rows2["patient_state"].unique()[0]
        b.loc[null_mask, "Region"] = rows2["Region"].unique()[0]
        b.loc[null_mask, "Division"] = rows2["patient_state"].unique()[0]

In [159]:
test = pd.concat([test.drop(b.index), b])

In [160]:
test[test.isnull().any(axis=1)]

,patient_id,patient_state,patient_zip3,patient_age,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,Region,Division,population,density,income_individual_median,labor_force_participation,unemployment_rate,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,commute_time,health_uninsured,veteran
5773,452914,NaN,645,41,C50412,Malig neoplasm of upper-outer quadrant of left...,C773,NaN,NaN,14063.16667,437.033333,28515.83333,61.166667,5.366667,83.95,5.35,1.316667,0.4,0.666667,2.733333,5.583333,7.45,17.233333,16.866667,12.533333,8.283333


In [161]:
test[test["patient_zip3"] == 645]

,patient_id,patient_state,patient_zip3,patient_age,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,Region,Division,population,density,income_individual_median,labor_force_participation,unemployment_rate,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple,hispanic,disabled,commute_time,health_uninsured,veteran
5773,452914,NaN,645,41,C50412,Malig neoplasm of upper-outer quadrant of left...,C773,NaN,NaN,14063.16667,437.033333,28515.83333,61.166667,5.366667,83.95,5.35,1.316667,0.4,0.666667,2.733333,5.583333,7.45,17.233333,16.866667,12.533333,8.283333


In [162]:
test.at[5773, 'patient_state'] = 'A'
test.at[5773, 'Region'] = 'B'
test.at[5773, 'Division'] = 'C'

Veri setinde kategorik değişkenler bulunduran sütunları sayısal değerlere dönüştürmek

In [163]:
merged_data_2 = pd.concat([training, test], axis=0)

In [164]:
categorical_columns = [
    'patient_state', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 
    'metastatic_cancer_diagnosis_code', 'Region', 'Division']
label_encoders = {}
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
for column in categorical_columns:
    if column in training.columns and column in test.columns: 
        merged_data_2[column] = label_encoders[column].fit_transform(merged_data_2[column].astype(str))

In [165]:
training = merged_data_2.iloc[:len(training), :]
test = merged_data_2.iloc[len(training):, :]

In [166]:
test = test.drop(["DiagPeriodL90D"], axis=1)

Modeli test ederken kullanmak için training verimizi test ve train olarak ikiye ayırıyoruz

In [167]:
X = training.drop(columns=['DiagPeriodL90D'])
y = training['DiagPeriodL90D']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

En doğru model ve parametrelere karar verme

In [168]:
model_lgbm = LGBMRegressor(n_estimators=100, random_state=42, colsample_bytree=0.5, 
                             learning_rate=0.01, max_depth=7, subsample=0.5)
model_lgbm.fit(X_train, y_train)
prediction_lgbm = model_lgbm.predict(X_test)
auc_lgbm = roc_auc_score(y_test, prediction_lgbm)
print("LightGBM AUC:", auc_lgbm)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001517 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7496
[LightGBM] [Info] Number of data points in the train set: 10324, number of used features: 36
[LightGBM] [Info] Start training from score 0.626792
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
LightGBM AUC: 0.7992932909219159
